### Quantum Teleportation Simulation using Qiskit Aer Simulator

In [25]:
# Import necessary libraries
from qiskit import QuantumCircuit, transpile, ClassicalRegister, QuantumRegister
from qiskit_aer import aer_simulator
from qiskit.quantum_info import Statevector, state_fidelity
import numpy as np 

In [26]:
# Function to prepare q0 state 
def prep_state_for_q0(qc, label):
    if(label == '|0>'):
        return
    elif(label == '|1>'):
        qc.x(0)  
    elif(label == '|+>'):
        qc.h(0)   
    elif(label == '|->'):
        qc.h(0)
        qc.z(0)   
    elif(label == '|i>'):
        qc.h(0)
        qc.s(0)
    elif(label == '|-i>'):
        qc.h(0)
        qc.sdg(0)
    else:
        raise ValueError('Unknown state label')
    return

In [27]:
# Function for creating quantum teleportation circuit
def qt_circuit(label):

    qr = QuantumRegister(3, 'q')
    cr = ClassicalRegister(3, 'c')
    qc = QuantumCircuit(qr, cr)

    prep_state_for_q0(qc, label)
    qc.barrier

    # Prepare entangle qubits at q1 and q2
    qc.h(1)
    qc.cx(1,2)

    # Gates for bell bases measurement
    qc.cx(0,1)
    qc.h(0)

    # Measure qubits in classical registers
    qc.measure([0,1], [0,1])
    qc.barrier

    # Apply Z gate and X gate to Bob's qubit according to cr[0], cr[1]
    # Instead of classical control, use conditional operations
    with qc.if_test((cr[1], 1)):
        qc.x(2)
    with qc.if_test((cr[0], 1)):
        qc.z(2)

    # Measure Bob's qubit
    qc.measure(2,2)

    return qc

In [ ]:
# Function to run simulation
def qt_simulation(label, shots = 1024):
    qc = qt_circuit(label)

    simulator = aer_simulator.AerSimulator()

    job = simulator.run(qc, shots = shots)
    result = job.result()
    counts = result.get_counts()

    return qc, counts

In [29]:
# Evaluate results
def qt_evaluate(counts):
    print("***Evaluation***")
    print(f"Total number of shots during simulation: {sum(counts.values())}")

    # Get Bob counts
    Bob_counts = {}

    for bits, shots in counts.items():
        Bob_bit = bits[0]

        if Bob_bit not in Bob_counts.keys():
            Bob_counts[Bob_bit] = 0
            Bob_counts[Bob_bit] += shots
        else:
            Bob_counts[Bob_bit] += shots

    return Bob_counts

In [30]:
# Function to calculate fidality
def qt_fidelity(label, Bob_counts, total_shots):
    # Expected probability of bits based on state label
    if label == '|0>':
        expected_prob = {'0':1.0, '1':0.0}
    elif label == '|1>':
        expected_prob = {'0':0.0, '1':1.0}
    else:      
        expected_prob = {'0':0.5, '1':0.5}
    
    # Actual probability of bits based on state label
    actual_prob = {}
    for bit in ['0', '1']:
        if bit in Bob_counts.keys():
            actual_prob[bit] = Bob_counts[bit]/total_shots
        else:
            actual_prob[bit] = 0.0
    
    # Calculate fidality
    fidelity = sum(np.sqrt(expected_prob[bit] * actual_prob[bit]) for bit in ['0', '1'])**2

    print(f"Expected probability of bits based on state label {label}:")
    print(expected_prob)
    print(f"Actual probability of bits after teleportation:")
    print(actual_prob)
    print(f"Fidelity: {fidelity}")


In [31]:
# Main function to run the quantum teleportation simulation
from sympy import evaluate


def main():
    print("******QUANTUM TELEPORTATION******")

    # Assign label
    valid_label = ['|0>', '|1>', '|+>', '|->', '|i>', '|-i>']

    flag = True
    while flag:
        state_label = input("Enter the qubit state:").strip()

        if state_label in valid_label:
            flag = False
        else:
            print(f"Please enter a qubit state from {','.join(valid_label)}.")
    
    # Assign number of shots
    while True:
        try:
            shots = int(input("Enter the total number of shots (by default 1024):"))
            if shots > 0:
                break
            elif shots < 0:
                print("Please enter a positive number.")
        except ValueError:
            print("Invalid total number of shots entered!")

    print(f"***Simulating Quantum Teleportation for qubit in state {state_label} with {shots} shots***")
    print()
    qc, counts = qt_simulation(state_label, shots)

    print("Quantum Circuit:")
    print()
    print(qc.draw())

    print("Result measurements corresponding to total number of shots is:")
    print(counts)

    Bob_counts = qt_evaluate(counts)
    print("Measured qubit teleported to Bob corresponding to total number of shots:")
    print(Bob_counts)

    qt_fidelity(state_label, Bob_counts, shots)
    


In [32]:
if __name__ == "__main__":
    main()

******QUANTUM TELEPORTATION******
***Simulating Quantum Teleportation for qubit in state |+> with 2000 shots***

Quantum Circuit:

     ┌───┐          ┌───┐┌─┐                                                   »
q_0: ┤ H ├───────■──┤ H ├┤M├───────────────────────────────────────────────────»
     ├───┤     ┌─┴─┐└┬─┬┘└╥┘                                                   »
q_1: ┤ H ├──■──┤ X ├─┤M├──╫────────────────────────────────────────────────────»
     └───┘┌─┴─┐└───┘ └╥┘  ║   ┌──────  ┌───┐ ───────┐   ┌──────  ┌───┐ ───────┐»
q_2: ─────┤ X ├───────╫───╫───┤ If-0  ─┤ X ├  End-0 ├───┤ If-0  ─┤ Z ├  End-0 ├»
          └───┘       ║   ║   └──╥───  └───┘ ───────┘   └──╥───  └───┘ ───────┘»
                      ║   ║ ┌────╨────┐               ┌────╨────┐              »
c: 3/═════════════════╩═══╩═╡ c_1=0x1 ╞═══════════════╡ c_0=0x1 ╞══════════════»
                      1   0 └─────────┘               └─────────┘              »
«         
«q_0: ────
«         
«q_1: ────
«      ┌─┐
«q_2